In [2]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go

In [3]:
data = pd.read_csv("dataset_PCG 2.txt", sep='\t', header=None)

# Extract time (x-axis) and signal (y-axis)
x = data[0]
y = data[1]

# Create a plotly scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Signal'))

# Customize the layout
fig.update_layout(
    title="Signal vs Time",
    xaxis_title="Time (s)",
    yaxis_title="Signal",
    template="plotly"
)

In [4]:
coloumncount = 100
rowcount = 100
a = 0.0001
da = 0.0001
dt = 1/8000
f0 = 0.849
w0 = 2*np.pi* f0
Ndata = len(y)
pi = np.pi

db = (Ndata - 1) * dt / coloumncount
cwtre = np.zeros((coloumncount, rowcount))
cwtim = np.zeros((coloumncount, rowcount))
cwt = np.zeros((coloumncount, rowcount))

db = (Ndata-1)* dt / coloumncount
for i in range(rowcount):
    b = 0.0
    for j in range(coloumncount):
        t = 0.0
        cwtre_sum = 0.0
        cwtim_sum = 0.0
        for k in range(Ndata):
            rem = (1 / np.sqrt(a)) * (1 / np.power(pi, 0.25)) * np.exp(-((t - b) / a) ** 2 / 2.0) * np.cos(w0 * (t - b) / a)
            imm = (1 / np.sqrt(a)) * (-1 / np.power(pi, 0.25)) * np.exp(-((t - b) / a) ** 2 / 2.0) * np.sin(w0 * (t - b) / a)
            cwtre_sum += y[k] * rem
            cwtim_sum += y[k] * imm
            t += dt
        
        cwtre[j, i] = cwtre_sum
        cwtim[j, i] = cwtim_sum
        cwt[j, i] = np.sqrt(cwtre[j, i] ** 2 + cwtim[j, i] ** 2)
        b += db
    
    a += da

In [5]:
X, Y = np.meshgrid(np.arange(rowcount), np.arange(coloumncount))
Z = cwt.T  

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')])
fig.update_layout(title='Continuous Wavelet Transform (CWT) 3D Plot',
                  scene=dict(xaxis_title='Scale',
                             yaxis_title='Freq',
                             zaxis_title='Magnitude'),
                  autosize=True)
fig.show()